https://www.kaggle.com/code/tommy1028/lightgbm-starter-with-feature-engineering-idea

Feature Engineering
Here are my thoughts on feature engieering with my background knowledge on financial market.

price_spread: the difference between ask price and bid price. Wide spread means low liquidity, leading to high volatility.
volume: the sum of the ask/bid size. Low volume means low liquidity, leading to high volatility
volume_imbalance: the difference between ask size and bid size. Large imbalance means low liquidity for one side, leading to high volatility

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)

import os
import glob


In [9]:
# data directory
data_dir = '/Users/shujinhuang/Downloads/'
df = pd.read_parquet(data_dir + "stock_id=15")
df.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999519,0.999839,0.999454,0.999904,2,166,2,12
1,5,1,0.999711,1.000225,0.999647,1.000289,100,20,100,20
2,5,2,0.999775,1.000225,0.999711,1.000289,1,20,400,20
3,5,3,0.999839,1.000225,0.999775,1.000289,100,20,1,20
4,5,4,0.999839,1.000225,0.999711,1.000289,1,20,400,20


### Preprocessing book data

In [7]:
def calc_wap(df):
    # 100 2 , 900 3, 2.9, 2.1 
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [15]:
def log_return(list_stock_prices):
    # np.log()
    return np.log(list_stock_prices).diff()  # diff() = later value - former value

In [21]:
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [23]:
df['wap'] = calc_wap(df)
df["log_return"] = df.groupby('time_id')['wap'].apply(log_return)

df['wap2'] = calc_wap2(df)
df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)

df['wap_balance'] = abs(df['wap'] - df['wap2'])

create_feature_dict = {
    'log_return':[realized_volatility],
    'log_return2':[realized_volatility],
#     'wap_balance':[np.mean],
#     'price_spread':[np.mean],
#     'bid_spread':[np.mean],
#     'ask_spread':[np.mean],
#     'volume_imbalance':[np.mean],
#     'total_volume':[np.mean],
    'wap':[np.mean],
        }
df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_

In [26]:
df.groupby(['time_id']).agg(create_feature_dict).reset_index()

,time_id,log_return,log_return2,wap
,,realized_volatility,realized_volatility,mean
0,5,0.004373,0.006178,1.002162
1,11,0.001772,0.002515,1.000220
2,16,0.002624,0.003697,0.999829
3,31,0.002814,0.004609,0.998441
4,62,0.001545,0.002619,0.998095
...,...,...,...,...
3825,32751,0.003185,0.004807,0.999413
3826,32753,0.002219,0.002938,0.999869
3827,32758,0.002112,0.002499,1.000331


In [ ]:
df.loc[df.time_id==32767]


In [27]:
df_feature.columns

MultiIndex([(    'time_id',                    ''),
            ( 'log_return', 'realized_volatility'),
            ('log_return2', 'realized_volatility'),
            (        'wap',                'mean')],
           )

### Preprocessing trade data

In [28]:
# create a name space
from enum import IntEnum

In [29]:
class Demo(IntEnum):
    Name = 0
    Age = 1

In [30]:
Demo.Name

<Demo.Name: 0>

In [31]:
from collections import namedtuple

In [32]:
Student = namedtuple("Student", ['name', 'age', 'sex', 'email'])

In [35]:
s = Student("Jim", 22, "M", "xxx")

In [36]:
s[0]

'Jim'

In [37]:
s.name

'Jim'